# European Engine

In [55]:
from QuantLib import *
import datetime
import re
import utils 
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from yahoo_fin.stock_info import get_quote_table
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
sys.path.insert(0, '../scripts/')
import utils as ut
from database import Stock 
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
from pandas.tseries.offsets import BDay
end = pd.datetime.today().date()
start = end - 252 * BDay() * 5 # five year historical data

In [56]:
np.std(np.log(Stock("AAPL", start, end).df.Close.pct_change() + 1)) * 252 ** 0.5

0.30467116819018114

Building the option requires only the specification of its contract, so its payoff (it's a call option with strike at 100) and its exercise, three months from today's date. Market data will be selected and passed later, depending on the calculation methods.

In [57]:
def european_engine(ticker, option="Call"):
                
    info = get_quote_table(ticker)
    current_price = info["Quote Price"]
    yield_re = re.compile(r"\((?P<value>(\d+\.\d+))%\)")
    try:
        dividend_rate = float(yield_re.search(info["Forward Dividend & Yield"])["value"])
    except (KeyError, ValueError, TypeError):
        dividend_rate = 0.0      
    
    today = Date.todaysDate()
    Settings.instance().evaluationDate = today
    
    stk_sigma = np.std(np.log(Stock("AAPL", start, end).df.Close.pct_change() + 1)) * 252 ** 0.5

    def create_call(row):
        u = SimpleQuote(current_price)
        r = SimpleQuote(0.001)
        sigma = SimpleQuote(stk_sigma)
        option = EuropeanOption(PlainVanillaPayoff(Option.Call, row["strike"]), EuropeanExercise(Date(expiration.day, expiration.month, expiration.year)))
        riskFreeCurve = FlatForward(0, TARGET(), QuoteHandle(r), Actual360())
        volatility = BlackConstantVol(0, TARGET(), QuoteHandle(sigma), Actual360())
        process = BlackScholesProcess(QuoteHandle(u),YieldTermStructureHandle(riskFreeCurve),BlackVolTermStructureHandle(volatility))
        process.dividendYield().zeroRate(dividend_rate, Continuous)
        
        engine = AnalyticEuropeanEngine(process)
        option.setPricingEngine(engine)
        
        npv = option.NPV()
        delta = option.delta()
        gamma = option.gamma()
        theta = option.theta()
        vega = option.vega()
        rho = option.rho()
        
        return {"Underlying": current_price,
                "Ticker": ticker,
                "Type": row["type"],
                "Expiration": row['expirationDate'],
                "YTE": row["yte"],
                "DTE": row["dte"],
                "Strike": row["strike"],
                "Last": row["lastPrice"],
                "Bid": row["bid"],
                "Ask": row["ask"],
                "Midpoint": (row['bid'] + row['ask']) / 2,
                "Spread": row['ask'] - row['bid'],
                "IV": row["impliedVolatility"],  
                "NPV": npv,
                "Delta": delta,
                "Gamma": gamma,
                "Theta":  theta / 365,
                "Vega": vega,
                "Rho": rho,
                "AKA": row["contractSymbol"], 
                "Intrinsic_Value":  current_price - row["strike"],
                "Time_Value": row["lastPrice"] - (current_price- row["strike"])}
        
    def create_put(row):
        u = SimpleQuote(current_price)
        r = SimpleQuote(0.001)
        sigma = SimpleQuote(stk_sigma)
        option = EuropeanOption(PlainVanillaPayoff(Option.Put, row["strike"]), EuropeanExercise(Date(expiration.day, expiration.month, expiration.year)))
        riskFreeCurve = FlatForward(0, TARGET(), QuoteHandle(r), Actual360())
        volatility = BlackConstantVol(0, TARGET(), QuoteHandle(sigma), Actual360())
        process = BlackScholesProcess(QuoteHandle(u),YieldTermStructureHandle(riskFreeCurve),BlackVolTermStructureHandle(volatility))
        process.dividendYield().zeroRate(dividend_rate, Continuous)
        engine = AnalyticEuropeanEngine(process)
        option.setPricingEngine(engine)
        npv = option.NPV()
        delta = option.delta()
        gamma = option.gamma()
        theta = option.theta()
        vega = option.vega()
        rho = option.rho()
        
        return {"Underlying": current_price,
                "Ticker": ticker,
                "Type": row["type"],
                "Expiration": row['expirationDate'],
                "YTE": row["yte"],
                "DTE": row["dte"],
                "Strike": row["strike"],
                "Last": row["lastPrice"],
                "Bid": row["bid"],
                "Ask": row["ask"],
                "Midpoint": (row['bid'] + row['ask']) / 2,
                "Spread": row['ask'] - row['bid'],
                "IV": row["impliedVolatility"],  
                "NPV": npv,
                "Delta": delta,
                "Gamma": gamma,
                "Theta":  theta / 365,
                "Vega": vega,
                "Rho": rho,
                "AKA": row["contractSymbol"], 
                "Intrinsic_Value":  row["strike"] - current_price, 
                "Time_Value": row["lastPrice"] - (row["strike"] - current_price)}          
        
    options_= pd.DataFrame()       
    
    if option == "Call":
        
        tk = yf.Ticker(ticker)
        exps = tk.options
        
        for e in exps:
            opt = tk.option_chain(e)
            calls = pd.DataFrame().append(opt.calls)
            calls['expirationDate'] = e
            expiration =  pd.to_datetime(e) + datetime.timedelta(days = 1)
            calls['expirationDate'] = expiration
            calls['yte'] = (calls['expirationDate'] - datetime.datetime.today()).dt.days / 365
            calls['dte'] = (calls['expirationDate'] - datetime.datetime.today()).dt.days
            calls[['bid', 'ask', 'strike']] = calls[['bid', 'ask', 'strike']].apply(pd.to_numeric)
            calls['type'] = "Call"
            calls = calls.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate'])
            options = calls.apply(create_call, axis=1, result_type="expand")
            options_ = options_.append(options, ignore_index=True) 
    else:    
        
        tk = yf.Ticker(ticker)
        exps = tk.options
        
        for e in exps:
            opt = tk.option_chain(e)
            puts = pd.DataFrame().append(opt.puts)
            puts['expirationDate'] = e
            expiration =  pd.to_datetime(e) + datetime.timedelta(days = 1)
            puts['expirationDate'] = expiration
            puts['yte'] = (puts['expirationDate'] - datetime.datetime.today()).dt.days / 365
            puts['dte'] = (puts['expirationDate'] - datetime.datetime.today()).dt.days
            puts[['bid', 'ask', 'strike']] = puts[['bid', 'ask', 'strike']].apply(pd.to_numeric)
            puts['type'] = "Put"
            puts = puts.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate'])
            options = puts.apply(create_put, axis=1, result_type="expand")
            options_ = options_.append(options, ignore_index=True) 
    return  options_

In [58]:
call = european_engine("AAPL", option="Call")

In [59]:
call.Last

0      62.7500
1      57.5000
2      52.4500
3      48.4000
4      39.5400
         ...  
1148   11.5900
1149   10.6000
1150    9.6000
1151    8.7500
1152    8.0000
Name: Last, Length: 1153, dtype: float64

In [60]:
call.NPV

0      63.0007
1      58.0007
2      53.0008
3      48.0008
4      43.0009
         ...  
1148   11.0676
1149   10.0600
1150    9.1435
1151    8.3102
1152    7.5530
Name: NPV, Length: 1153, dtype: float64

In [61]:
put = european_engine("AAPL", option="Put")

In [63]:
put.Last

0       0.0100
1       0.0100
2       0.0100
3       0.0100
4       0.0100
         ...  
1103   23.4000
1104   25.5700
1105   29.4800
1106   39.6500
1107   63.9300
Name: Last, Length: 1108, dtype: float64

In [64]:
put.NPV

0      -0.0000
1      -0.0000
2      -0.0000
3      -0.0000
4      -0.0000
         ...  
1103   23.1495
1104   26.2345
1105   29.4783
1106   40.0556
1107   64.1524
Name: NPV, Length: 1108, dtype: float64